In [24]:
import os
import numpy as np
import json
import scipy.io as sio
import torch
from sklearn.preprocessing import MultiLabelBinarizer
import pickle

Import data

In [25]:
def importDataFromMatFiles(rootDir):
    """
    Param: rootDir: The parent directory to the directories that start with Copy ....
                    For example, in this case, rootDir = "/Users/jameshe/Documents/radar_ura/FirstBatch"
    """
    xList = list()
    yList = list()
    mlb = MultiLabelBinarizer()
    for f in os.scandir(rootDir):
        if f.is_dir():
            with open(os.path.join(f.path, "test_data.json")) as labelFile:
                labels = json.load(labelFile)
                occupancyLabel = labels["Occupied_Seats"]
            for file in os.scandir(os.path.join(f.path, "SavedVars_RF_Data")):
                frame = sio.loadmat(file)
                image = frame["Image"]
                mask = frame["Mask"]
                image[mask == False] = 0
                xList.append(image)
                yList.append(occupancyLabel)
    yList = mlb.fit_transform(yList)
    xList = np.array(xList)
    xList = np.absolute(xList)
    return (xList, yList)

In [26]:
def saveData(data, path, fileName):
    """
    open file mode w: write, if the file exist, erase it
    open file mode b: open the file as a binary file
    """
    filePath = os.path.join(path, fileName)
    with open(filePath,'wb') as pickleFileHandle:
        pickle.dump(data, pickleFileHandle)
        pickleFileHandle.close()

In [27]:
def loadData(path, fileName):
    """
    open file mode b: open the file as a binary file
    open file mode r: read file
    """
    filePath = os.path.join(path, fileName)
    with open(filePath, 'rb') as pickleFileHandle:
        data = pickle.load(pickleFileHandle)
        return data

In [28]:
x, y = importDataFromMatFiles("/Users/jameshe/Documents/radar_ura/FirstBatch")